In [10]:
import gc
import itertools
import matplotlib.pyplot as plt
import numpy as np
import random

import pandas as pd
import rpy2.rinterface as rinterface
import rpy2.robjects as robjects

import tqdm
import seaborn as sns
import scipy.stats as stats

from itertools import compress
from Bio import motifs
from Bio.Seq import Seq #, IUPAC
from collections import Counter
from os import listdir
from os.path import join
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

from scipy.stats import ks_2samp
from statistics import mean, median

%load_ext rpy2.ipython
%matplotlib inline
pandas2ri.activate()
plt.ioff()

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [2]:
# Import R packages and define python objects to use in R
bs_genome = importr('BSgenome.Hsapiens.UCSC.hg19')
bio_strings = importr('Biostrings')
genomic_ranges = importr('GenomicRanges')


## Functions

In [39]:
def score_fwd_rc_seq_simple(dicti,path_to_jaspar_db):
    max_scores_for_motifs = {}
    with open(path_to_jaspar_db) as handle:
        for m in tqdm.tqdm(motifs.parse(handle, 'jaspar')):
            m.pseudocounts = 0.5  # Add pseudocounts to Position Frequency Matrix m
            motif_name = m.name.split('.')[0].split('_')[0]
            pssm = m.pssm  # Get Position-Specific Scoring Matrix
            rc_pssm = m.pssm.reverse_complement()  # Get reverse compliment to Position-Specific Scoring Matrix
            max_scores_for_sequence = []
            for seq in dicti.values():
                # Select for sequences at least 2 times longer than a motif
                fwd_sequence_scores = pssm.calculate(Seq(seq))
                rc_sequence_scores = rc_pssm.calculate(Seq(seq))
                fwd_rc_max_list =  np.maximum(fwd_sequence_scores, rc_sequence_scores)
                max_scores_for_sequence.append(fwd_rc_max_list)
            max_scores_for_motifs[motif_name] = max_scores_for_sequence
    return(max_scores_for_motifs)

def edit_sequences(dict_with_sequences,
                   peak_info_dictionary):
    
    print('Editing sequences...')
    dict_with_sequences_for_ref_and_alt = {}
    for peak, sequence in dict_with_sequences.items():
        peak_id = int(peak)
        snp_position = int(peak_info_dictionary[peak_id]['Pos_shifted'])
        reference = peak_info_dictionary[peak_id]['Ref']
        alternative = peak_info_dictionary[peak_id]['Alt']
        ref_and_alt_dict = {}
        if len(reference.split(',')) > 1 or len(alternative.split(',')) > 1:
            # If there are several variants -- iterate over all possibile pairs
            ref_sequences, alt_sequences = [], []
            list_with_all_possible_pairs = list(itertools.product(reference.split(','),
                                                                  alternative.split(',')))
            for i, pair in enumerate(list_with_all_possible_pairs):
                ref_sequences.append(sequence[:snp_position]
                                     + sequence[snp_position :
                                                snp_position + len(pair[0])
                                               ].replace(reference, pair[0])
                                     + sequence[snp_position + len(reference):])
                alt_sequences.append(sequence[:snp_position]
                                     + sequence[snp_position :
                                                snp_position + len(pair[0])
                                               ].replace(reference, pair[1])
                                     + sequence[snp_position + len(reference):])

            ref_and_alt_dict['Ref_' + str(peak_id)] = list(set(ref_sequences))
            ref_and_alt_dict['Alt_' + str(peak_id)] = list(set(alt_sequences))
        else:
            ref_and_alt_dict['Ref_' + str(peak_id)] = [sequence]                             
            ref_and_alt_dict['Alt_' + str(peak_id)] = [sequence[:snp_position]
                                                  + sequence[snp_position :
                                                             snp_position + len(reference)
                                                            ].replace(reference, alternative)\
                                                  + sequence[snp_position + len(reference):]]
        dict_with_sequences_for_ref_and_alt[peak] = ref_and_alt_dict
    return dict_with_sequences_for_ref_and_alt


def create_groups_of_sequences_ref_alt(dict_with_edited_ref_alt_sequences):
    
    print('Splitting into two groups according reference and alt...')
    dict_with_sequences_for_ref_group = {}
    dict_with_sequences_for_alt_group = {}
    for peak, ref_alt_sequence_dict in dict_with_edited_ref_alt_sequences.items():
        peak_id = str(peak)
        dict_with_sequences_for_ref_group['ref_' + peak_id] = ref_alt_sequence_dict['Ref_' + peak_id][0]
        dict_with_sequences_for_alt_group['alt_' + peak_id] = ref_alt_sequence_dict['Alt_' + peak_id][0]
    return dict_with_sequences_for_ref_group, dict_with_sequences_for_alt_group


def extract_toppos_fwd_rc_seq_simple(dicti,path_to_jaspar_db):
    max_pos_for_motifs = {}
    with open(path_to_jaspar_db) as handle:
        for m in tqdm.tqdm(motifs.parse(handle, 'jaspar')):
            m.pseudocounts = 0.5  # Add pseudocounts to Position Frequency Matrix m
            motif_name = m.name.split('.')[0].split('_')[0]
            pssm = m.pssm  # Get Position-Specific Scoring Matrix
            rc_pssm = m.pssm.reverse_complement()  # Get reverse compliment to Position-Specific Scoring Matrix
            toppos_for_sequence = []
            for seq in dicti.values():
                # Select for sequences at least 2 times longer than a motif
                fwd_sequence_scores = pssm.calculate(Seq(seq))
                rc_sequence_scores = rc_pssm.calculate(Seq(seq))
                fwd_rc_max_list =  np.maximum(fwd_sequence_scores, rc_sequence_scores)
                top_pos = np.argmax(fwd_rc_max_list)
                toppos_for_sequence.append(top_pos)
            max_pos_for_motifs[motif_name] = toppos_for_sequence
    
    return(max_pos_for_motifs)

def get_df_with_topscore(scores_dict):
    top_score_dict = {}
    for TF,enh in scores_dict.items():
        max_score=[]
        for scores in enh:
            max_score.append(-np.sort(-scores)[0])
        top_score_dict[TF]=max_score
    return(top_score_dict)



In [4]:
# load motifs
dict_with_motifs = create_dict_with_motif_lengths('/../HOCOMOCOv11_core_HUMAN_mono_jaspar_format.txt')


100%|██████████| 401/401 [00:00<00:00, 859435.82it/s]


In [6]:
## peak table
sum_tab = pd.read_csv('/../unique_Leads_Master_table_LEAD_DEP_pairs_noncau_local_vcm_AllspInfo_2022_10_24_v1.3.csv') # final Master table provided with publication


sum_tab['UID'] = np.array(range(sum_tab.shape[0]))
sum_tab

,Unnamed: 0,Peak,Chr,Pos,RsID,Ref,Alt,AF,Inside_Peak,P_Lead,...,depGMSNP_altcount,ATAC_lead_ref,ATAC_lead_alt,ATAC_lead_act,ATAC_lead_pas,ATAC_dep_ref,ATAC_dep_alt,ATAC_dep_act,ATAC_dep_pas,UID
0,7,7,chr1,565286,rs1578391,C,T,0.59780,1,0.998561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,222,213,chr1,2508343,rs200553981,G,C,0.35000,1,0.999924,...,13.0,6.0,10.0,10.0,6.0,12.0,13.0,12.0,13.0,1
2,287,277,chr1,3820005,rs10909828,G,A,0.34500,1,0.999966,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,296,286,chr1,4327635,rs2101576,C,T,0.55000,1,0.999782,...,NaN,69.0,1.0,69.0,1.0,NaN,NaN,NaN,NaN,3
4,404,390,chr1,6837155,rs74050914,T,C,0.13500,1,0.805640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17282,29564,27393,chr2,45755773,rs3821060,T,A,0.29500,1,0.450728,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17282
17283,30357,28147,chr2,53317253,rs80099213,C,T,0.08000,1,0.330063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17283
17284,31476,29217,chr2,62408114,rs13004362,G,C,0.28005,1,0.265662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17284
17285,32257,29972,chr2,70237472,rs11126258,G,T,0.06500,1,0.386771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17285


In [7]:
# prep for scoring
peaks_dom = sum_tab.loc[:,['Chr','Pos_Left','Pos_Right','Peak','Peak_center','Alt','Ref','Beta','Pos']]

peaks_dom.index = peaks_dom['Peak']

dom_to_score = peaks_dom.drop_duplicates()
#dom_to_score = dom_to_score.iloc[:10,:]
dom_to_score.columns = ["chr","start","end","Peak","center",'Alt','Ref','Beta','Pos']
dom_to_score['start'] = dom_to_score['center']-350
dom_to_score['end'] = dom_to_score['center']+350

print(dom_to_score.shape,sum_tab.shape)
dom_to_score


(17287, 9) (17287, 93)


,chr,start,end,Peak,center,Alt,Ref,Beta,Pos
Peak,,,,,,,,,
7,chr1,565046,565746,7,565396,T,C,0.886884,565286
213,chr1,2507870,2508570,213,2508220,C,G,0.226942,2508343
277,chr1,3819736,3820436,277,3820086,A,G,-0.654595,3820005
286,chr1,4327368,4328068,286,4327718,T,C,-0.666789,4327635
390,chr1,6836772,6837472,390,6837122,C,T,-0.528177,6837155
...,...,...,...,...,...,...,...,...,...
27393,chr2,45755576,45756276,27393,45755926,A,T,-0.068858,45755773
28147,chr2,53316706,53317406,28147,53317056,T,C,-0.053419,53317253
29217,chr2,62407928,62408628,29217,62408278,C,G,0.033513,62408114


In [28]:
#R seqeunce extraction
robjects.globalenv['dom_to_score'] = dom_to_score.iloc[0:10,:] # example first 10
GRanges_for_dom_peaks = genomic_ranges.makeGRangesFromDataFrame(dom_to_score.iloc[0:10,:])
sequences_for_dom_peaks = bio_strings.getSeq(bs_genome.Hsapiens, GRanges_for_dom_peaks)
robjects.globalenv['sequences_for_dom_peaks'] = sequences_for_dom_peaks

with localconverter(robjects.default_converter + pandas2ri.converter):
  pandas_dataframe = robjects.conversion.rpy2py(robjects.r('as.data.frame(sequences_for_dom_peaks)'))

dict_with_sequences_for_dom_peaks = pandas_dataframe.to_dict()['x']
dict_with_sequences_for_dom_peaks

{'7': 'CTACTCTACCATCTTTGCAGGCACACTCATCACAGCGCTAAGCTCGCACTGATTTTTTACCTGAGTAGGCCTAGAAATAAACATGCTAGCTTTTATTCCAGTTCTAACCAAAAAAATAAACCCTCGTTCCACAGAAGCTGCCATCAAGTATTTCCTCACGCAAGCAACCGCATCCATAATCCTTCTAATAGCTATCCTCTTCAACAATATACTCTCCGGACAATGAACCATAACCAATACCACCAATCAATACTCATCATTAATAATCATAATGGCTATAGCAATAAAACTAGGAATAGCCCCCTTTCACTTCTGAGTCCCAGAGGTTACCCAAGGCACCCCTCTGACATCCGGCCTGCTCCTTCTCACATGACAAAAACTAGCCCCCATCTCAATCATATACCAAATTTCTCCCTCATTAAACGTAAGCCTTCTCCTCACTCTTTCAATCTTATCCATCATGGCAGGCAGTTGAGGTGGATTAAACCAAACCCAACTACGCAAAATCTTAGCATACTCCTCAATTACCCACATAGGATGAATAACAGCAGTTCTACCGTACAACCCTAACATAACCATTCTTAATTTAACTATTTATATTATCCTAACTACTACCGCATTCCTACTACTCAACTTAAACTCCAGCACCACAACCCTACTACTATCTCGCACCTGAAACAAGCTAACATGACTAACACCCT',
 '213': 'TTGGGTCCAGAACTGTCTCGGCTGGAATTCGGGTTCCCGGCGTGGCTCCACCAGGGCAGCAGCCCCAGAGGCAAAGAGGTCTCCTTCTCCGGCTGAGCTCATCCCTGGGGCTGAACTTTGCTGAGAGGATCAGCTCCCAGGCCTGGCCAGCCCTCCAGTGAGCACCAACTACGCTGGCTGCAGGGAGTGCTTGTGTCTGAAGAACTCCAGGCTGGTGTGCCCAGAGGGCGGCTGGAGGGGCTCAGAAACCATTTTCACTTTTCTGTCTCCTCCATGTCTG

In [29]:
dom_to_score['Pos_shifted'] = dom_to_score['Pos'] - dom_to_score['start']
dict_with_dom_peak_info = dom_to_score[['Pos_shifted','Ref','Alt']].T.to_dict('dict')
dict_with_dom_peak_info

# edit ref seqeunces to also create seqeunces with alternative genotypes
sequences_for_dom_ref_and_alt = edit_sequences(dict_with_sequences_for_dom_peaks,dict_with_dom_peak_info)

ref_seqs_dom, alt_seqs_dom = create_groups_of_sequences_ref_alt(sequences_for_dom_ref_and_alt)


Editing sequences...
Splitting into two groups according reference and alt...


In [23]:
ref_seqs_dom['ref_7']

'CTACTCTACCATCTTTGCAGGCACACTCATCACAGCGCTAAGCTCGCACTGATTTTTTACCTGAGTAGGCCTAGAAATAAACATGCTAGCTTTTATTCCAGTTCTAACCAAAAAAATAAACCCTCGTTCCACAGAAGCTGCCATCAAGTATTTCCTCACGCAAGCAACCGCATCCATAATCCTTCTAATAGCTATCCTCTTCAACAATATACTCTCCGGACAATGAACCATAACCAATACCACCAATCAATACTCATCATTAATAATCATAATGGCTATAGCAATAAAACTAGGAATAGCCCCCTTTCACTTCTGAGTCCCAGAGGTTACCCAAGGCACCCCTCTGACATCCGGCCTGCTCCTTCTCACATGACAAAAACTAGCCCCCATCTCAATCATATACCAAATTTCTCCCTCATTAAACGTAAGCCTTCTCCTCACTCTTTCAATCTTATCCATCATGGCAGGCAGTTGAGGTGGATTAAACCAAACCCAACTACGCAAAATCTTAGCATACTCCTCAATTACCCACATAGGATGAATAACAGCAGTTCTACCGTACAACCCTAACATAACCATTCTTAATTTAACTATTTATATTATCCTAACTACTACCGCATTCCTACTACTCAACTTAAACTCCAGCACCACAACCCTACTACTATCTCGCACCTGAAACAAGCTAACATGACTAACACCCT'

### extract top score for each TF in ehancers

In [36]:
scores_dom_ref = score_fwd_rc_seq_simple(ref_seqs_dom,'/../HOCOMOCOv11_core_HUMAN_mono_jaspar_format.txt')
scores_dom_alt = score_fwd_rc_seq_simple(alt_seqs_dom,'/../HOCOMOCOv11_core_HUMAN_mono_jaspar_format.txt')


100%|██████████| 401/401 [00:01<00:00, 319.48it/s]


In [38]:
top_score_ref_dict = get_df_with_topscore(scores_dom_ref)
top_score_alt_dict = get_df_with_topscore(scores_dom_alt)

dom_df_top_refsc = pd.DataFrame.from_dict(top_score_ref_dict)
dom_df_top_refsc.index = dom_to_score['Peak'].iloc[0:10]

dom_df_top_altsc = pd.DataFrame.from_dict(top_score_alt_dict)
dom_df_top_altsc.index = dom_to_score['Peak'].iloc[0:10]

dom_df_top_altsc

,AHR,AIRE,ALX1,ANDR,AP2A,AP2B,AP2C,ARI5B,ARNT,ASCL1,...,ZN768,ZN770,ZN816,ZNF18,ZNF41,ZNF76,ZNF85,ZNF8,ZSC22,ZSC31
Peak,,,,,,,,,,,,,,,,,,,,,
7,10.910510,16.992689,11.113787,7.646778,6.015738,8.379262,5.640248,11.053022,7.799374,6.629828,...,0.510246,1.958955,2.062617,5.877241,3.300840,4.758318,7.430340,1.149459,10.459830,6.476737
213,10.206508,11.569746,3.765735,4.443468,12.859211,11.656336,14.119665,6.436944,9.910398,10.425703,...,10.808111,12.071813,5.594294,5.815779,3.728640,12.158457,4.792959,-5.294492,13.392704,13.020210
277,4.755745,8.346975,8.231889,7.410455,8.138215,8.988652,7.786445,5.994706,5.035826,14.814066,...,12.812613,10.162467,8.059190,5.233010,10.247425,5.717460,4.686571,-1.441987,11.549913,11.832839
286,7.728946,9.111926,4.767227,10.410053,9.138053,7.693197,8.546309,11.641253,6.835958,7.289433,...,14.184060,16.527033,11.644974,6.737186,9.594412,8.657131,4.988376,-9.806476,8.554649,15.049697
390,11.504558,7.461088,10.175416,6.955098,7.097834,5.351569,6.675338,9.565577,7.190668,14.752342,...,9.953867,23.572084,5.979954,5.746092,5.832504,9.217098,5.584718,-4.382401,12.776009,9.853341
549,8.911577,7.614105,7.832562,6.629333,10.183543,8.234872,10.434277,9.565577,8.774660,10.186225,...,1.600030,21.334188,2.117108,7.374807,4.863650,8.344782,9.830227,-7.736543,10.649038,11.663212
571,9.039991,7.632474,6.709740,10.550818,9.626948,7.239925,10.636164,7.555624,5.771461,12.051066,...,4.687716,5.527819,8.647938,6.251130,11.778420,16.184345,7.100932,-3.609653,9.306936,8.181977
678,11.267259,6.088368,6.261921,3.510044,10.428874,11.867334,11.311463,7.038623,12.742774,4.105349,...,2.304117,4.212773,6.884624,8.749179,5.729620,7.103232,5.885442,-10.230060,10.427959,9.665303
884,9.163579,11.187135,10.718558,7.470758,6.217303,7.955896,4.699228,9.513148,7.432683,5.859718,...,4.688565,4.240335,8.021681,6.007191,4.795929,10.636304,2.169095,-0.672643,13.187113,7.539889


### extract top score position for each TF in ehancers

In [30]:
top_pos_dom_ref = extract_toppos_fwd_rc_seq_simple(ref_seqs_dom,'/../HOCOMOCOv11_core_HUMAN_mono_jaspar_format.txt')
top_pos_dom_alt = extract_toppos_fwd_rc_seq_simple(alt_seqs_dom,'../HOCOMOCOv11_core_HUMAN_mono_jaspar_format.txt')


100%|██████████| 401/401 [00:01<00:00, 315.04it/s]


In [33]:
topsc_dom_df_ref=pd.DataFrame(top_pos_dom_ref)
topsc_dom_df_ref['Peak']=ref_seqs_dom.keys()
topsc_dom_df_alt = pd.DataFrame(top_pos_dom_alt)
topsc_dom_df_alt['Peak']=alt_seqs_dom.keys()
#print(topsc_dom_df_ref)
topsc_dom_df_alt

,AHR,AIRE,ALX1,ANDR,AP2A,AP2B,AP2C,ARI5B,ARNT,ASCL1,...,ZN770,ZN816,ZNF18,ZNF41,ZNF76,ZNF85,ZNF8,ZSC22,ZSC31,Peak
0,155,229,582,581,325,338,315,232,154,131,...,462,402,97,79,309,175,385,354,345,alt_7
1,595,542,282,293,99,454,371,19,594,491,...,452,483,213,582,172,188,33,449,672,alt_213
2,20,620,663,21,325,83,326,12,464,530,...,622,163,357,311,119,272,45,564,539,alt_277
3,110,427,343,566,306,521,177,439,109,406,...,62,58,102,602,499,274,338,65,199,alt_286
4,589,547,228,533,386,504,387,562,590,344,...,618,91,401,33,318,2,142,646,260,alt_390
5,604,554,281,13,244,123,245,185,605,639,...,115,457,158,116,266,149,108,659,56,alt_549
6,203,480,280,679,376,18,377,5,204,512,...,69,423,13,313,375,172,209,121,399,alt_571
7,446,650,677,634,32,324,375,653,199,375,...,332,326,166,468,318,298,9,70,540,alt_678
8,560,389,395,118,10,326,10,579,559,186,...,486,479,265,595,93,502,283,234,432,alt_884
9,473,150,150,181,626,628,627,685,679,524,...,198,345,436,7,65,260,515,623,641,alt_1028


#### for enhancers with no SNPs only reference sequences have to be scored